In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

In [2]:
# 设置超参数
d = 20
alpha_u = alpha_v = alpha_w = beta_u = beta_v = 0.01
lr = 0.01
epochs = 100

user_num = 943
item_num = 1682
n = 90570

In [3]:
# 读取数据
train_data = pd.read_csv('/Users/chao/workspace/d2l/data/ml-100k/ua.base', sep='\t', names=['userId', 'itemId', 'rating', 'timestamp'])
test_data = pd.read_csv('/Users/chao/workspace/d2l/data/ml-100k/ua.test', sep='\t', names=['userId', 'itemId', 'rating', 'timestamp'])

# 随机打乱数据
def shuffle_data(data):
    indices = list(range(len(data)))
    random.shuffle(indices)
    return data.iloc[indices,:]

In [4]:
# 打乱训练集
train_data = shuffle_data(train_data)
explicit_data = train_data.iloc[0:int(n/2),:]
implicit_data = train_data.iloc[int(n/2):,:]

# 生成显示反馈
ratings = np.zeros((user_num + 1, item_num + 1), int)
y_ui = np.zeros((user_num + 1, item_num + 1), int)
for i in range(len(explicit_data)):
    userId, itemId, rating = explicit_data.iloc[i,:]['userId'], explicit_data.iloc[i,:]['itemId'], explicit_data.iloc[i,:]['rating']
    ratings[userId][itemId] = rating
    y_ui[userId][itemId] = 1

# 生成隐式反馈
feedbacks = np.zeros((user_num + 1, item_num + 1), int)
for i in range(len(implicit_data)):
    userId, itemId, rating = implicit_data.iloc[i,:]['userId'], implicit_data.iloc[i,:]['itemId'], implicit_data.iloc[i,:]['rating']
    feedbacks[userId][itemId] = 1

# 求用户u在隐式反馈表中评分过的所有物品
def I_bar(u):
    return np.where(feedbacks[u] == 1)[0]

# 求用户u在W上的偏好
def U_bar(u, W):
    items = I_bar(u)
    return (np.sum(W[items], axis=0) / np.sqrt(len(items))) if len(items) != 0 else np.zeros(d)

In [5]:
# 初始化参数
def init():
    mu = (y_ui * ratings).sum() / y_ui.sum()
    bu = np.zeros(user_num + 1, float)
    for u in range(1, user_num + 1):
        bu[u] = (0 if y_ui[u].sum() == 0 else (y_ui[u] * (ratings[u] - mu)).sum() / y_ui[u].sum())
    bi = np.zeros(item_num + 1, float)
    for i in range(1, item_num + 1):
        bi[i] = (0 if y_ui[:,i].sum() == 0 else (y_ui[:,i] * (ratings[:,i] - mu)).sum() / y_ui[:,i].sum())
    U = np.random.rand(user_num + 1, d)
    V = np.random.rand(item_num + 1, d)
    W = np.random.rand(item_num + 1, d)
    U = (U - 0.5) * 0.01
    V = (V - 0.5) * 0.01
    W = (W - 0.5) * 0.01
    return mu, bu, bi, U, V, W

# 预测函数
def predict_rule(u, i, mu, bu, bi, U, V, W):
    return U[u] @ V[i].T + U_bar(u, W) @ V[i].T + bu[u] + bi[i]+ mu

In [6]:
mu, bu, bi, U, V, W = init()
for epoch in tqdm(range(epochs)):
    # 打乱训练集
    explicit_data = shuffle_data(explicit_data)
    for t in range(len(explicit_data)):
        # 随机取数据
        u, i, rating = explicit_data.iloc[t,:]['userId'], explicit_data.iloc[t,:]['itemId'], explicit_data.iloc[t,:]['rating']
        # 计算梯度
        e = ratings[u][i] - predict_rule(u, i, mu, bu, bi, U, V, W)
        delta_mu = -e
        delta_bu = -e + beta_u * bu[u]
        delta_bi = -e + beta_v * bi[i]
        delta_Uu = (-e) * V[i] + alpha_u * U[u]
        delta_Vi = (-e) * (U[u] + U_bar(u, W)) + alpha_v * V[i]
        items = I_bar(u)
        delta_W = np.zeros_like(W)
        if len(items) != 0:
            delta_W[items] = -e / np.sqrt(len(items)) * V[i] + alpha_w * W[items]
        # 更新参数
        mu -= lr * delta_mu
        bu[u] -= lr * delta_bu
        bi[i] -= lr * delta_bi
        U[u] -= lr * delta_Uu
        V[i] -= lr * delta_Vi
        W[items] -= lr * delta_W[items]
    # 学习率下降
    lr *= 0.9

100%|█████████████████████████████████████████| 100/100 [07:21<00:00,  4.42s/it]


In [7]:
# 预测规则
def SVD_PP(u, j):
    return predict_rule(u, j, mu, bu, bi, U, V, W)

In [8]:
# 损失函数
def MAE(predict_rule):
    data_num = test_data.shape[0]
    loss = 0.0
    for i in range(data_num):
        userId, itemId, rating = test_data.iloc[i,:]['userId'], test_data.iloc[i,:]['itemId'], test_data.iloc[i,:]['rating'] 
        y_hat = postProcess(predict_rule(userId, itemId))
        loss += abs(y_hat - rating)
    return loss / data_num

def RMSE(predict_rule):
    data_num = test_data.shape[0]
    loss = 0.0
    for i in range(data_num):
        userId, itemId, rating = test_data.iloc[i,:]['userId'], test_data.iloc[i,:]['itemId'], test_data.iloc[i,:]['rating'] 
        y_hat = postProcess(predict_rule(userId, itemId))
        loss += ((y_hat - rating) ** 2) / data_num
    return loss ** 0.5

# 数据后处理
def postProcess(num):
    num = min(5.0, num)
    num = max(1.0, num)
    return num

# 预测
def predict(*predict_rules):
    for predict_rule in predict_rules:
        print(f"RMSE: {RMSE(predict_rule):.4f}, MAE: {MAE(predict_rule):.4f}")

In [9]:
# 输出结果
predict(SVD_PP)

RMSE: 0.9966, MAE: 0.7810
